<a href="https://colab.research.google.com/github/LuisMirandaMallqui/Inteligencia_Artificial_TareaAcademica_Grupo8_H783/blob/main/Modular_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

¡Claro que sí\! He revisado tu notebook (`Grupo8-TA-Final_v2.pdf`) y veo que has hecho un gran trabajo en la carga, renombrado y limpieza inicial. Estás justo en el punto más importante: **cómo estructurar el preprocesamiento y el modelado**.

Tu profesor tiene toda la razón. El "PELIGRO" que anotas en tu PDF es real: aplicar normalización y encoding a todo el dataset *antes* de dividirlo es un error común llamado **"fuga de datos" (data leakage)**.

La solución es el flujo que te recomendé. He reestructurado tu notebook en un script modular y lógico que sigue esta metodología. Este código está diseñado para que lo copies y pegues, reemplazando tu sección de preprocesamiento y modelado.

-----

## **Flujo de Trabajo Corregido y Modular (Notebook Completo)**

Este código toma tu limpieza, la organiza y la integra con los `Pipelines` y el `ColumnTransformer` que hemos discutido, respetando las indicaciones de tu profesor.

In [ ]:
# ==============================================================================
# 1. LIBRERÍAS
# ==============================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Métricas
from sklearn.metrics import mean_squared_error, r2_score

# Preprocesamiento
from sklearn.model_selection import train_test_split # Lo usaremos, pero cronológicamente
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Configuración
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 7)
print("Librerías importadas.")

# ==============================================================================
# 2. CARGA Y RENOMBRADO DE DATOS
# ==============================================================================
print("\n--- 2. Cargando y Renombrando Datos ---")
try:
    # Cargamos especificando dtypes de columnas problemáticas como 'str'
    df_nacimientos = pd.read_csv(
        'CNV_MINSA_4782338_CORTE_310825.csv',
        delimiter=';',
        on_bad_lines='skip',
        dtype={
            'Num_embar_madre': 'str',
            'Hijos_vivo_madre': 'str',
            'Hijos_fallec_madre': 'str',
            'nacmuer_abort_madre': 'str'
        }
    )
    df_ubigeos = pd.read_csv('Lista_Ubigeos_INEI.csv', delimiter=';')
    print("Archivos cargados exitosamente.")
except FileNotFoundError:
    print("Error: Asegúrate de que los archivos CSV estén en la ubicación correcta.")
    # (Detener la ejecución o manejar el error)

# Diccionario de renombrado (tomado de tu notebook)
df_nacimientos.rename(columns={
    'FecNac_Año': 'Año_Nacimiento',
    'FecNac_Mes': 'Mes_Nacimiento',
    'PESO_NACIDO': 'Peso_Nacido',
    'TALLA_NACIDO': 'Talla_Nacido',
    'DUR_EMB_PARTO': 'Duracion_Embarazo',
    'Condicion_Parto': 'Condicion_Parto',
    'sexo_nacido': 'Sexo_Nacido',
    'Tipo_Parto': 'Tipo_Parto',
    'Edad_Madre': 'Edad_Madre',
    'Estado_Civil': 'Estado_Civil_Madre',
    'Nivel_Intrucción_Madre': 'Nivel_Instruccion_Madre',
    'DESC_OCUPACION': 'Ocupacion_Madre',
    'Num_embar_madre': 'Numero_Embarazos_Madre',
    'Hijos_vivo_madre': 'Hijos_Vivos_Madre',
    'Hijos_fallec_madre': 'Hijos_Fallecidos_Madre',
    'nacmuer_abort_madre': 'Abortos_Madre',
    'Pais_Madre': 'Pais_Madre',
    'IdUbigeoInei': 'Codigo_Ubigeo',
    'Ipress': 'Ipress_Hospital',
    'Lugar_Nacido': 'Lugar_Nacimiento',
    'Atiende_Parto': 'Atiende_Parto',
    'Financiador_Parto': 'Financiador_Parto'
}, inplace=True)
print("Columnas renombradas.")

# ==============================================================================
# 3. FUNCIÓN DE LIMPIEZA DE COLUMNAS NUMÉRICAS
# ==============================================================================
def limpiar_columna_conteo(columna, mapeo_extra={}):
    """
    Limpia una columna de conteo (como 'Hijos') que viene como texto.
    Convierte 'NINGUNO', 'IGNORADO', '-1' y mapeos extra a valores numéricos.
    """
    # Mapeo base
    mapeo_base = {
        'NINGUNO': 0,
        'IGNORADO': 0, # Decidimos tratar 'IGNORADO' como 0
        '-1': 0        # Tratar -1 como 0
    }

    # Combinar con mapeos específicos (ej. '11 a más': 11)
    mapeo_total = {**mapeo_base, **mapeo_extra}

    columna_limpia = columna.replace(mapeo_total)

    # Convertir a numérico, todo lo que no sea número se vuelve NaN
    columna_num = pd.to_numeric(columna_limpia, errors='coerce')

    # Rellenar NaNs (los 'coerce' y los originales) con 0
    columna_final = columna_num.fillna(0)

    return columna_final.astype(int)

print("\n--- 3. Limpiando y Validando Datos Individuales ---")

# --- 3.1. Aplicar limpieza a las columnas de conteo ---
df_nacimientos['Numero_Embarazos_Madre'] = limpiar_columna_conteo(
    df_nacimientos['Numero_Embarazos_Madre'], {'>=5': 5}
)
df_nacimientos['Hijos_Vivos_Madre'] = limpiar_columna_conteo(
    df_nacimientos['Hijos_Vivos_Madre'], {'11 a mas': 11}
)
df_nacimientos['Abortos_Madre'] = limpiar_columna_conteo(
    df_nacimientos['Abortos_Madre'], {'11 a más': 11} # Corregido
)
print("Columnas de conteo (embarazos, hijos, abortos) limpiadas.")

# --- 3.2. Validación de Rangos Biológicos ---
registros_antes = len(df_nacimientos)
df_nacimientos = df_nacimientos[
    (df_nacimientos['Edad_Madre'] >= 12) & (df_nacimientos['Edad_Madre'] <= 55) &
    (df_nacimientos['Peso_Nacido'] >= 500) & (df_nacimientos['Peso_Nacido'] <= 5000) &
    (df_nacimientos['Talla_Nacido'] >= 30) & (df_nacimientos['Talla_Nacido'] <= 60)
]
registros_despues = len(df_nacimientos)
print(f"Se eliminaron {registros_antes - registros_despues} registros por rangos biológicos inválidos.")

# --- 3.3. Merge Geográfico ---
df_nacimientos['Codigo_Ubigeo'] = pd.to_numeric(df_nacimientos['Codigo_Ubigeo'], errors='coerce')
df_ubigeos['UBIGEO_INEI'] = pd.to_numeric(df_ubigeos['UBIGEO_INEI'], errors='coerce')
df_completo = pd.merge(
    df_nacimientos,
    df_ubigeos,
    left_on='Codigo_Ubigeo',
    right_on='UBIGEO_INEI',
    how='left'
)
# Es crucial eliminar filas que no tengan departamento para la agregación
df_completo.dropna(subset=['DEPARTAMENTO'], inplace=True)
print("Merge geográfico realizado.")

# ==============================================================================
# 4. AGREGACIÓN TRIMESTRAL (Creación del Dataset para el Modelo)
# ==============================================================================
print("\n--- 4. Agregando datos por Trimestre y Departamento ---")
df_completo['Trimestre'] = pd.to_datetime(df_completo['Mes_Nacimiento'], format='%m').dt.quarter
df_completo['Año_Nacimiento'] = pd.to_numeric(df_completo['Año_Nacimiento'], errors='coerce')
df_completo.dropna(subset=['Año_Nacimiento'], inplace=True)
df_completo['Año_Nacimiento'] = df_completo['Año_Nacimiento'].astype(int)

# Función para obtener la moda (el valor más frecuente)
def obtener_moda(x):
    return x.mode().get(0, np.nan)

df_agregado = df_completo.groupby(['Año_Nacimiento', 'Trimestre', 'DEPARTAMENTO']).agg(
    # Variable Objetivo (y)
    Nacimientos_Totales=('Codigo_Ubigeo', 'count'),

    # Variables Predictoras (X)
    Edad_Madre_Promedio=('Edad_Madre', 'mean'),
    Peso_Promedio=('Peso_Nacido', 'mean'),
    Talla_Promedio=('Talla_Nacido', 'mean'),
    Hijos_Vivos_Promedio=('Hijos_Vivos_Madre', 'mean'),
    Num_Embarazos_Promedio=('Numero_Embarazos_Madre', 'mean'),
    Abortos_Promedio=('Abortos_Madre', 'mean'),
    Nivel_Educativo_Moda=('Nivel_Instruccion_Madre', obtener_moda),
    Estado_Civil_Moda=('Estado_Civil_Madre', obtener_moda)
).reset_index()

print(f"Dataset agregado creado con {len(df_agregado)} filas (Trimestre-Departamento).")

# ==============================================================================
# 5. DIVISIÓN DE DATOS (Train-Test Split) ¡ANTES DE PROCESAR!
# ==============================================================================
print("\n--- 5. Dividiendo datos en Entrenamiento y Prueba ---")
# Como dijo tu profesor: división cronológica
# Entrenamos con datos hasta 2022, probamos con 2023 en adelante.

frontera_año = 2022
df_train = df_agregado[df_agregado['Año_Nacimiento'] <= frontera_año]
df_test = df_agregado[df_agregado['Año_Nacimiento'] > frontera_año]

print(f"Registros de Entrenamiento (<= {frontera_año}): {len(df_train)}")
print(f"Registros de Prueba (> {frontera_año}): {len(df_test)}")

# --- 5.1. Definir Features (X) y Target (y) ---
TARGET = 'Nacimientos_Totales'

# Definimos nuestras listas de columnas
numeric_features = [
    'Año_Nacimiento', 'Edad_Madre_Promedio', 'Peso_Promedio',
    'Talla_Promedio', 'Hijos_Vivos_Promedio', 'Num_Embarazos_Promedio',
    'Abortos_Promedio'
]
categorical_features = [
    'Trimestre', 'DEPARTAMENTO', 'Nivel_Educativo_Moda', 'Estado_Civil_Moda'
]

X_train = df_train[numeric_features + categorical_features]
y_train = df_train[TARGET]

X_test = df_test[numeric_features + categorical_features]
y_test = df_test[TARGET]

# ==============================================================================
# 6. DEFINICIÓN DEL PIPELINE DE PREPROCESAMIENTO
# ==============================================================================
print("\n--- 6. Definiendo Pipelines de Preprocesamiento ---")
# Esto resuelve tu "PELIGRO" de qué hacer con las columnas
# Cada grupo se trata por separado.

# Pipeline para variables NUMÉRICAS
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Rellena NaNs con la mediana
    ('scaler', MinMaxScaler()) # Normaliza los datos (escala 0-1)
])

# Pipeline para variables CATEGÓRICAS
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Rellena NaNs con la moda
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # Crea variables dummy
])

# Usamos ColumnTransformer para aplicar los pipelines a las columnas correctas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# ==============================================================================
# 7. ENTRENAMIENTO Y EVALUACIÓN DE MODELOS
# ==============================================================================
print("\n--- 7. Entrenando y Evaluando Modelos ---")

# --- 7.1. Definir los modelos que pidió tu profesor ---
modelos = {
    "Regresión Lineal": LinearRegression(),
    "Árbol de Decisión": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1)
}

resultados = []

for nombre, modelo_base in modelos.items():

    # 1. Crear el Pipeline COMPLETO (Preprocesar -> Modelar)
    pipeline_modelo = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', modelo_base)
    ])

    # 2. Entrenar el pipeline SOLO con datos de TRAIN
    print(f"Entrenando {nombre}...")
    pipeline_modelo.fit(X_train, y_train)

    # 3. Evaluar el pipeline SOLO con datos de TEST
    y_pred = pipeline_modelo.predict(X_test)

    # 4. Calcular Métricas
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    resultados.append({
        "Modelo": nombre,
        "R² (Test)": r2,
        "RMSE (Test)": rmse
    })
    print(f"Resultados de {nombre}: R² = {r2:.4f}, RMSE = {rmse:.4f}\n")

# --- 7.2. Tabla Comparativa de Resultados ---
df_resultados = pd.DataFrame(resultados).set_index("Modelo")
print("\n--- TABLA COMPARATIVA DE MÉTRICAS (R² y RMSE) ---")
print(df_resultados)
print("\n(R² cercano a 1 es mejor. RMSE cercano a 0 es mejor)")

# ==============================================================================
# 8. ANÁLISIS DE IMPORTANCIA DE VARIABLES (Random Forest)
# ==============================================================================
print("\n--- 8. Analizando Importancia de Variables (Random Forest) ---")

# Re-entrenamos el pipeline de Random Forest para extraer los nombres
# (Ya estaba entrenado, pero lo hacemos explícito para más claridad)
pipeline_rf = modelos["Random Forest"] # Modelo base
pipeline_completo_rf = Pipeline(steps=[('preprocessor', preprocessor), ('model', pipeline_rf)])
pipeline_completo_rf.fit(X_train, y_train)

# Obtener los nombres de las features después del OneHotEncoding
try:
    nombres_cat = pipeline_completo_rf.named_steps['preprocessor'] \
                 .named_transformers_['cat'] \
                 .named_steps['onehot'] \
                 .get_feature_names_out(categorical_features)

    # Todos los nombres de features
    nombres_features = numeric_features + list(nombres_cat)

    # Obtener las importancias
    importancias = pipeline_completo_rf.named_steps['model'].feature_importances_

    # Crear un DataFrame de importancias
    df_importancia = pd.DataFrame({
        'Variable': nombres_features,
        'Importancia': importancias
    }).sort_values(by='Importancia', ascending=False)

    print("\n--- Top 15 Variables Más Influyentes ---")
    print(df_importancia.head(15))

    # --- Gráfico de Importancia ---
    plt.figure(figsize=(12, 8))
    sns.barplot(
        data=df_importancia.head(15),
        x='Importancia',
        y='Variable',
        palette='viridis'
    )
    plt.title('Top 15 Variables Más Influyentes (Random Forest)')
    plt.show()

except Exception as e:
    print(f"Error al obtener importancia de variables: {e}")
    print("Asegúrate de que el modelo esté entrenado.")

print("\n--- ANÁLISIS COMPLETADO ---")